# 自定义梯度计算

有时候仅仅依靠back propagation自动求梯度并不会得到正确的结果，因为计算机运算存在不稳定的问题，比如浮点数溢出等。当计算不稳定的时候，可以使用tf.custom_gradient自定义梯度计算公式。

tf.custom_gradient在eager模式和graph模式中都可以使用

In [5]:
import tensorflow as tf
from tensorflow.contrib import eager as tfe

In [2]:
tf.enable_eager_execution()

比如计算$\ln(1+e^x)$这个操作

In [4]:
def log1pexp(x):
    return tf.log(1+tf.exp(x))

In [6]:
grad = tfe.gradients_function(log1pexp)

In [7]:
grad(0.)[0].numpy()

0.5

当x=100的时候计算基础

In [8]:
grad(100.)[0].numpy()

nan

$$
    \frac{\partial \ln (1 + e ^ x) }{ \partial x} = 1 - \frac{1}{1 + e ^ x}
$$

使用tf.custom_gradient自定义计算梯度公式

In [10]:
@tf.custom_gradient
def custom_log1pexp(x):
    e = tf.exp(x)
    def grad(dy):
        return dy * (1 - 1 / (1 + e))
    return tf.log(1 + e), grad

e = tf.exp(x)会在前向forward的过程中计算，backward的时候可以直接使用。dy是backward的时候之前所有操作的梯度，比如:
$$
    y = h(g(x))
$$
梯度：
$$
\frac{\partial y}{\partial x} = \frac{\partial h}{ \partial g} \frac{\partial g}{ \partial x}
$$
假设h就是custom_log1pexp，dy就是$\partial g / \partial x$

In [11]:
grad_custom = tfe.gradients_function(custom_log1pexp)

In [12]:
grad_custom(0.)[0].numpy()

0.5

In [13]:
grad_custom(100.)[0].numpy()

1.0

使用tf.custom_gradient进行梯度裁剪

In [ ]:
@tf.custom_gradient
def clip_gradient_by_norm(x, norm):
  y = tf.identity(x) # 返回和x一样的内容
  def grad_fn(dresult):
    return [tf.clip_by_norm(dresult, norm), None] # 对norm没有梯度，返回none
  return y, grad_fn